In [1]:
import xarray as xr
import hvplot.xarray

# Explore SM2RAIN-ASCAT dataset

In this notebook we explore the SM2RAIN-ASCAT dataset v2.1.2n from Brocca et al. [Brocca et al. 10.5281/zenodo.10376109](https://zenodo.org/records/10376109) (see corresponding [publication](https://essd.copernicus.org/articles/11/1583/2019/)). See [`download-data.bsh`](../download-data.bsh) for how to retrieve (part of) the dataset.

In [2]:
# define bounding box
lon_min = -74
lon_max = -70
lat_min = -42
lat_max = -38

In [3]:
# open and concatenate NetCDF files
ds = xr.open_mfdataset(
    "../data/SM2RAIN_ASCAT_01_v2-1-2n_year_*.nc", 
    chunks="auto", 
    engine="h5netcdf",
)

In [4]:
# extract rainfall, crop to bounding box 
rainfall = ds["rainfall"] \
  .sel(lon=slice(lon_min, lon_max), 
       lat=slice(lat_max, lat_min)
  )

In [5]:
# aggregations
rainfall_yearly = rainfall.resample(time="YS").sum().load()
rainfall_monthly = rainfall.resample(time="MS").sum().load()

# update attributes
rainfall_yearly.attrs["long_name"] = "Yearly rainfall in mm"
rainfall_monthly.attrs["long_name"] = "Monthly rainfall in mm"

In [6]:
def plot(dataarray):
    """ Generate plot """
    # set 0 to NaN
    dataarray = dataarray.where(dataarray > 0)
    # extract 98th percentile, set as max value for colorbar
    q98 = dataarray.quantile(0.98)
    return dataarray.hvplot.quadmesh(
        tiles=True, 
        alpha=0.6, 
        frame_height=600,
        frame_width=600,
        widget_location='bottom',
        clim=(0, q98),
    )

In [7]:
plot_yearly = plot(rainfall_yearly)
plot_yearly

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom')
    [1] WidgetBox(align=('center', 'end'))
        [0] DiscreteSlider(name='time', options={'2007-01-01 00:00:00': np...}, value=np.datetime64('2007-01-01T...)

In [8]:
plot_monthly = plot(rainfall_monthly)
plot_monthly

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom')
    [1] WidgetBox(align=('center', 'end'))
        [0] DiscreteSlider(name='time', options={'2007-01-01 00:00:00': np...}, value=np.datetime64('2007-01-01T...)

In [9]:
# export
hvplot.save(plot_monthly, "rainfall_monthly.html", embed=True)
hvplot.save(plot_yearly, "rainfall_yearly.html", embed=True)